In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plot
from speccounts import *
from utilities import *



In [ ]:
def mangle_poly(templatespectrum,filter_file_list, zeropointlist, pivotlist, counts_in,n):

    input_wave,input_flux = clean_spectrum("../spectra/" + templatespectrum)#dtype=float,usecols=(0,1),unpack=True)
    clean_template = np.column_stack((input_wave,input_flux))

    #######  Calculate the counts in each filter from the template and compare to counts_in

    counts_array=get_counts_multi_filter(clean_template, filter_file_list)
    ratio = np.zeros(len(counts_array))

    for x in range(0,len(counts_array)):
        ratio[x]=counts_in[x]/counts_array[x]

    p=np.polyfit(pivotlist, ratio,n)
    manglefunction= np.polyval(p,input_wave)

    mangledspectrumflux=input_flux*manglefunction

    return input_wave, mangledspectrumflux


In [ ]:
def countrates2mags(output_file_name, template_spectrum):
    counts_df=pd.read_csv('../input/COUNTS/'+output_file_name+'_mangledcounts.csv')
    filter_bands= list(counts_df.columns[1:])
    filter_file_list, zeropointlist, pivotlist = filterlist_to_filterfiles(
       filter_bands , template_spectrum)
    for idx,zeropoint in enumerate(zeropointlist):
        counts_df[filter_bands[idx]]=counts_df[filter_bands[idx]].apply(lambda count: (math.log10(count)/-0.4)+zeropoint)
    counts_df.to_csv('../output/MAGS/'+output_file_name+'_mangledmagsarray.csv', index=False)